## Setup

In [3]:
import os
pwd = os.path.abspath('.')
os.chdir(os.path.join(pwd, '../../src/'))

In [4]:
from collections import Counter
import pandas as pd
import seaborn as sns
from core.environment import EnvironmentSettings
from string import Template
import matplotlib.pyplot as plt
import numpy as np

import datetime

In [5]:
%load_ext autoreload
%autoreload 2

## Analysis

### Setup

##### Import

In [6]:
from core.column_definition import DerivativesBaseColumns, DerivativesCalculatedColumns
from trade_builders.derivatives import DerivativeTradeOptions, OptionsMultiTradesBuilder, \
  StrategyAnalyser, DerivatesMultiTradeOptions, DerivativesNonTradablesDiagnostic
from trade_builders.results import TradesResult

In [7]:
from calculations.derivatives import DerivativesLotSizeCalculationWorker, DerivativesPriceCalculationWorker
from dataprocess.data_processor import CalculationPipeline

derivatives_calc_pipeline = CalculationPipeline()
derivatives_calc_pipeline.add_calculation_worker(DerivativesLotSizeCalculationWorker())
derivatives_calc_pipeline.add_calculation_worker(DerivativesPriceCalculationWorker())

##### Setup datareaders

In [8]:
from datareader.data_reader import NseDerivatiesReader, DateRangeDataReader, NseDerivatiesOldReader
reader = NseDerivatiesReader()
old_reader = NseDerivatiesOldReader()
daterange_reader = DateRangeDataReader(reader)
daterange_old_reader = DateRangeDataReader(old_reader)

yest = datetime.date.today() - datetime.timedelta(days=1)

##### Setup Trade Options

In [9]:
multi_trade_options_allow_overlap = DerivatesMultiTradeOptions()
multi_trade_options_allow_overlap.avoid_overlap_trades = False

trade_options_aggr = DerivativeTradeOptions()
trade_options_aggr.profit_factor = 3

trade_options_cons = DerivativeTradeOptions()
trade_options_cons.profit_factor = 1.5
trade_options_cons.loss_factor = 0.75

trade_options_patience = DerivativeTradeOptions()
trade_options_patience.holding_days = 14

trade_options_low_capital = DerivativeTradeOptions()
trade_options_low_capital.budget_min = 4000
trade_options_low_capital.budget_max = 8000

trade_options_high_capital = DerivativeTradeOptions()
trade_options_high_capital.budget_min = 12000
trade_options_high_capital.budget_max = 16000

trade_options_low_to_high_capital = DerivativeTradeOptions()
trade_options_low_to_high_capital.budget_min = 8000
trade_options_low_to_high_capital.budget_max = 16000

trade_options_very_low_to_high_capital = DerivativeTradeOptions()
trade_options_very_low_to_high_capital.budget_min = 2000
trade_options_very_low_to_high_capital.budget_max = 16000

trade_options_delayed_entry = DerivativeTradeOptions()
trade_options_delayed_entry.entry_lag_days = 1

# trade_options_delayed_entry_avg_price = DerivativeTradeOptions()
# trade_options_delayed_entry_avg_price.entry_lag_days = 1
# trade_options_delayed_entry_avg_price.entry_amount_col = DerivativesBaseColumns.AveragePrice

In [10]:
multi_trade_options = {
    'default': DerivatesMultiTradeOptions(),
    'allow_overlap': multi_trade_options_allow_overlap
}

trade_options = {
    'default': DerivativeTradeOptions(),
    # 'aggr': trade_options_aggr,
    # 'cons': trade_options_cons,
    # 'patience': trade_options_patience,
    # 'low_capital': trade_options_low_capital,
    # 'high_capital': trade_options_high_capital,
    'low_to_high_capital': trade_options_low_to_high_capital,
    # 'delayed_entry': trade_options_delayed_entry,
    'very_low_to_high_capital': trade_options_very_low_to_high_capital
}

### Overbought / Oversold Stocks

##### Get Strategy Results

In [11]:
output_sub_folder = 'trades/rsi-optimized'

In [13]:
strategy_results = []
for year in range(2016, 2023 + 1):
  print(f'Executing for {year}')
  options_data = daterange_old_reader.read(datetime.date(year, 1, 1), 
    min(datetime.datetime.today(), datetime.datetime(year, 12, 31)))
  options_data = derivatives_calc_pipeline.run(options_data)

  analyzer = StrategyAnalyser(output_sub_folder + '/' + str(year)).set_multi_trade_options(multi_trade_options).set_trade_options(trade_options)
  strategy_result = analyzer.analyze(options_data = options_data)
  strategy_results.append(result)

Executing for 2016
DerivativesLotSizeCalculationWorker took 1 seconds
DerivativesPriceCalculationWorker took 24 seconds
Executing for 2017
DerivativesLotSizeCalculationWorker took 1 seconds
DerivativesPriceCalculationWorker took 20 seconds
Executing for 2018
DerivativesLotSizeCalculationWorker took 1 seconds
DerivativesPriceCalculationWorker took 25 seconds
Executing for 2019
File is not a zip file date(2019, 02, 27),
File is not a zip file date(2019, 05, 09),
DerivativesLotSizeCalculationWorker took 1 seconds
DerivativesPriceCalculationWorker took 24 seconds
Executing for 2020
DerivativesLotSizeCalculationWorker took 1 seconds
DerivativesPriceCalculationWorker took 24 seconds
Executing for 2021
File is not a zip file date(2021, 03, 22),
https://archives.nseindia.com/content/historical/DERIVATIVES/2021/MAR/fo30MAR2021bhav.csv.zip
The read operation timed out date(2021, 03, 30),
[Errno 2] No such file or directory: '../../_data/raw/nse_derivatives/fo12MAY2021bhav/fo12MAY2021bhav.csv' da

In [17]:
strategy_results = []
for year in range(2016, 2023 + 1):
  analyzer = StrategyAnalyser(output_sub_folder + '/' + str(year)).set_multi_trade_options(multi_trade_options).set_trade_options(trade_options)
  result = analyzer.load_from_files()
  result.save_result(f'../output/{output_sub_folder}/{str(year)}/final.xlsx')
  strategy_results.append(result)

In [18]:
all_underlying_summaries = pd.DataFrame()
for year in range(2016, 2023 + 1):
  underlying_summaries = pd.read_excel(f'../output/{output_sub_folder}/{str(year)}/final.xlsx', 'underlying-summaries')
  underlying_summaries['year'] = year
  all_underlying_summaries = pd.concat([all_underlying_summaries, underlying_summaries])

In [19]:
out_cols = ['net_pnl', 'total_expense', 'no_of_trades', 'non_tradables_no_entry_count', 'non_tradables_no_exit_count']
all_underlying_summaries.groupby('id')[out_cols].sum().sort_values('net_pnl', ascending=False)

,net_pnl,total_expense,no_of_trades,non_tradables_no_entry_count,non_tradables_no_exit_count
id,,,,,
allow_overlap-low_to_high_capital,19280766,2782349,22256,6149,205
default-low_to_high_capital,18280269,2554689,20382,6102,204
allow_overlap-default,15503975,2081958,19140,7773,139
default-default,14537906,1925003,17678,7662,138
allow_overlap-very_low_to_high_capital,11670046,2046876,25780,4372,220
default-very_low_to_high_capital,11077862,1884545,23780,4344,218


In [21]:
method = 'allow_overlap-low_to_high_capital'
#method = 'allow_overlap-delayed_entry'
all_underlying_summaries.groupby('id').get_group(method)

,Unnamed: 0,break_even_date,brokerage,capital_required,net_pnl,no_of_trades,non_tradables_no_entry_count,non_tradables_no_exit_count,pnl,roi,total_expense,transaction_fees,turnover,id,year
4,4,2016-02-08,30680,215509,1284084,1534,1135,27,1471531,596,187447,185913,18591328,allow_overlap-low_to_high_capital,2016
4,4,2017-05-15,48240,261802,1810022,2412,1267,34,2108047,691,298025,295613,29561269,allow_overlap-low_to_high_capital,2017
4,4,2018-02-09,56680,553528,2739877,2834,1212,20,3087119,495,347242,344408,34440810,allow_overlap-low_to_high_capital,2018
4,4,2019-05-20,49680,315734,2187703,2484,979,37,2489171,693,301468,298984,29898370,allow_overlap-low_to_high_capital,2019
4,4,2020-02-05,47120,263895,2076838,2356,554,17,2375694,787,298856,296500,29650011,allow_overlap-low_to_high_capital,2020
4,4,2021-01-20,65000,920890,2881230,3250,381,29,3294913,313,413683,410433,41043337,allow_overlap-low_to_high_capital,2021
4,4,2022-01-20,82760,776275,4385782,4138,460,24,4916548,565,530766,526628,52662766,allow_overlap-low_to_high_capital,2022
4,4,2023-03-06,64960,414152,1915230,3248,161,17,2320092,462,404862,401614,40161446,allow_overlap-low_to_high_capital,2023


In [17]:
bear_full_trades = pd.read_excel('../output/trades/rsi/2023/default-delayed_entry.xlsx', 'bear-full-trades')

In [ ]:
from datareader.data_reader import BhavCopyReader
reader = BhavCopyReader()

from datareader.data_reader import DateRangeDataReader
from dataprocess.data_processor import HistoricalDataProcessor, MultiDataCalculationPipelines, CalculationPipelineBuilder
histDataProcessor = HistoricalDataProcessor()

pipelines = MultiDataCalculationPipelines()
pipelines.set_item('rsa', CalculationPipelineBuilder.create_rsa_calculation_pipeline())

histDataProcessor.set_calculation_pipelines(pipelines)

result = histDataProcessor.process(reader, {'from_date': datetime.date(2023, 1, 1), 'to_date': datetime.date.today()})
histDataProcessor.run_calculation_pipelines()
eqiuity_data = result.get_daily_data()

In [32]:
from core.column_definition import BaseColumns, CalculatedColumns

bear_full_trades_1 = pd.merge(bear_full_trades,eqiuity_data[[BaseColumns.Identifier, BaseColumns.Date, CalculatedColumns.RelativeStrengthIndex]], on=[BaseColumns.Identifier, BaseColumns.Date])

In [33]:
bear_full_trades_1

,Unnamed: 0,Identifier,StrkPric,ExpiryDate,OptionType,Date,Debit,ExitDate,Credit,Quantity,Turnover,Net,id,Rsi
0,0,SIEMENS,3100.0,29-Mar-2023,PE,2023-02-23,10298.75,2023-03-03,13433.75,275,23732.50,3135.00,bear,81.170989
1,1,HCLTECH,1100.0,23-Feb-2023,PE,2023-01-24,8820.00,2023-02-01,6545.00,700,15365.00,-2275.00,bear,78.779234
2,2,HCLTECH,1140.0,23-Feb-2023,PE,2023-02-02,11830.00,2023-02-10,23660.00,700,35490.00,11830.00,bear,76.641231
3,3,HCLTECH,1140.0,27-Jul-2023,PE,2023-07-03,10815.00,2023-07-10,21630.00,700,32445.00,10815.00,bear,76.018061
4,4,ABB,3000.0,29-Mar-2023,PE,2023-02-22,11212.50,2023-03-01,5606.25,250,16818.75,-5606.25,bear,76.390504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,418,BHARTIARTL,880.0,27-Jul-2023,PE,2023-07-12,8217.50,2023-07-18,16435.00,950,24652.50,8217.50,bear,80.362694
427,419,SAIL,85.0,27-Jul-2023,PE,2023-07-03,12000.00,2023-07-10,6000.00,8000,18000.00,-6000.00,bear,79.883946
428,420,SAIL,90.0,27-Jul-2023,PE,2023-07-17,11200.00,2023-07-21,5600.00,8000,16800.00,-5600.00,bear,82.474940
429,421,ZEEL,170.0,31-Aug-2023,PE,2023-07-12,11550.00,2023-07-13,0.00,3000,11550.00,-11550.00,bear,79.438735


In [36]:
from core.core import MarketDaysHelper
for_date = datetime.datetime(2023, 1, 20)
for i in range(0, 2):
  for_date = MarketDaysHelper.get_next_market_day(for_date)
for_date

datetime.datetime(2023, 1, 24, 0, 0)

#### Diagnose Non tradables

In [47]:
buy_signals = pd.read_csv('../output/trades/rsi/2023/buy_signal_symbols.csv')
sell_signals = pd.read_csv('../output/trades/rsi/2023/sell_signal_symbols.csv')

In [48]:
s = 'SIEMENS'
d = '2023-01-20'
print('Buy signals: ' + str(len(buy_signals[(buy_signals['Identifier'] == s) & (buy_signals['Date'] == d)])))
print('Sell signals: ' + str(len(sell_signals[(sell_signals['Identifier'] == s) & (sell_signals['Date'] == d)])))

Buy signals: 0
Sell signals: 1


In [54]:
year = 2023
analyzer = StrategyAnalyser(output_sub_folder + '/' + str(year)).set_multi_trade_options(multi_trade_options).set_trade_options(trade_options)
result = analyzer.load_from_files()
non_tradables = result.non_tradables
non_tradables = result.results['default-delayed_entry'].non_tradables

In [50]:
year = 2023
options_data = daterange_old_reader.read(datetime.date(year, 1, 1), datetime.date.today())
options_data = derivatives_calc_pipeline.run(options_data)
options_tradable_data = options_data[options_data[DerivativesBaseColumns.OpenInterest] > 0]

[Errno 2] No such file or directory: '../../_data/raw/nse_derivatives/fo13FEB2023bhav/fo13FEB2023bhav.csv' date(2023, 02, 13),
https://archives.nseindia.com/content/historical/DERIVATIVES/2023/AUG/fo10AUG2023bhav.csv.zip
DerivativesLotSizeCalculationWorker took 2 seconds
DerivativesPriceCalculationWorker took 45 seconds


In [90]:
from trade_builders.derivatives import DerivativesNonTradablesDiagnostic
from trade_builders.derivatives import BearOptionsTradeBuilder
bear_trade_builder = BearOptionsTradeBuilder(trade_options['delayed_entry'])

diag_options = DerivativesNonTradablesDiagnostic.EntryOptions()
from core.core import TypeHelper
props = ['skip_oi_check', 'skip_expiry_date_check', 'skip_budget_check'] #
#props = TypeHelper.get_class_props(diag_options)

In [60]:
index = 0
symbol = non_tradables[DerivativesBaseColumns.Identifier].values[index]
signal_date = non_tradables[DerivativesBaseColumns.Date].values[index]

In [ ]:
options_data[(options_data['Identifier'] == symbol) & (options_data['Date'] == bear_trade_builder.get_first_entry_date(signal_date)) & (options_data['OptionType'] == 'PE')]

In [91]:
diag_options = DerivativesNonTradablesDiagnostic.EntryOptions()
for prop in props:
    setattr(diag_options, prop, True)
    trade_option = bear_trade_builder.get_entry_option(options_data, symbol, signal_date, index = 0, diagnostic = diag_options)
    if len(trade_option) > 0:
      print(f'option available for {symbol} on {pd.to_datetime(trade_option[DerivativesBaseColumns.Date].values[0]).strftime("%d-%b-%y")} for {abs(round(trade_option["Credit"].values[0]))} with {prop} check')
      break

option available for SIEMENS on 23-Jan-23 for 9625 with skip_oi_check check


In [73]:
bear_trade_builder.get_first_entry_date(signal_date)

Timestamp('2023-01-23 00:00:00')

In [53]:
diagnostic = diag_options
hold_till_date = bear_trade_builder.get_last_exit_date(signal_date)
data = options_data
entry_option = data[
    (data[DerivativesBaseColumns.Identifier] == symbol)
    & (data[DerivativesBaseColumns.OptionType] == bear_trade_builder._option_type)
    & (data[DerivativesBaseColumns.Date] == bear_trade_builder.get_first_entry_date(signal_date))
    & (
        (data[bear_trade_builder._trade_options.entry_amount_col].between(4000, 12000))
        |
        (diagnostic.skip_budget_check == True)
      )
    & (
        (pd.to_datetime(data[DerivativesBaseColumns.ExpiryDate]) >= hold_till_date)
        |
        (diagnostic.skip_expiry_date_check == True)
      )
    & (
        (data[DerivativesBaseColumns.OpenInterest] >= (data[DerivativesCalculatedColumns.LotSize] * bear_trade_builder._trade_options.min_lots_oi))
        |
        (diagnostic.skip_oi_check == True)
      )
].sort_values(DerivativesBaseColumns.StrikePrice, ascending=False)
entry_option

,INSTRUMENT,Identifier,ExpiryDate,StrkPric,OptionType,Open,High,Low,Close,SETTLE_PR,...,PreviousClose,CloseToPrevCloseChangePerc,OpenToPrevCloseChangePerc,PreviousCloseAmount,OpenAmount,CloseAmount,HighAmount,LowAmount,AmountDiffOpenToPrevClose,AmountDiffCloseToPrevClose
130551,OPTSTK,HINDZINC,25-Feb-2016,180.0,PE,18.8,18.8,18.4,18.40,18.40,...,NaN,NaN,NaN,NaN,60160.0,58880.0,60160.0,58880.0,NaN,NaN
130550,OPTSTK,HINDZINC,25-Feb-2016,160.0,PE,0.0,0.0,0.0,6.55,7.10,...,6.55,0.000000,-100.000000,20960.0,0.0,20960.0,0.0,0.0,-20960.0,0.0
130549,OPTSTK,HINDZINC,25-Feb-2016,155.0,PE,3.7,4.5,3.7,3.80,3.80,...,4.30,-11.627907,-13.953488,13760.0,11840.0,12160.0,14400.0,11840.0,-1920.0,-1600.0
130548,OPTSTK,HINDZINC,25-Feb-2016,150.0,PE,4.0,4.0,4.0,4.00,4.00,...,2.20,81.818182,81.818182,7040.0,12800.0,12800.0,12800.0,12800.0,5760.0,5760.0
130547,OPTSTK,HINDZINC,25-Feb-2016,140.0,PE,0.0,0.0,0.0,0.85,1.05,...,0.85,0.000000,-100.000000,2720.0,0.0,2720.0,0.0,0.0,-2720.0,0.0
130546,OPTSTK,HINDZINC,25-Feb-2016,125.0,PE,0.0,0.0,0.0,2.25,0.10,...,2.25,0.000000,-100.000000,7200.0,0.0,7200.0,0.0,0.0,-7200.0,0.0


In [79]:

for index, row in non_tradables.head(5).iterrows():
  symbol = row[DerivativesBaseColumns.Identifier]
  signal_date = row[DerivativesBaseColumns.Date]
  for prop in props:
    diag_options = NonTradablesDiagnostic.EntryOptions()
    setattr(diag_options, prop, True)
    trade_option = trade_builder.get_entry_option(data_2023, symbol, signal_date, index = 0, diagnostic = diag_options)
    if len(trade_option) > 0:
      print(f'option available for {symbol} on {pd.to_datetime(trade_option[DerivativesBaseColumns.Date].values[0]).strftime("%d-%b-%y")} for {abs(round(trade_option["Credit"].values[0]))} with {prop} check')
      break

option available for SIEMENS on 24-Mar-23 for 9075 with skip_oi_check check
option available for HCLTECH on 30-Jan-23 for 8260 with skip_oi_check check
option available for INDIAMART on 17-Feb-23 for 9000 with skip_oi_check check


In [90]:
test_result = OptionsMultiTradesResult.load_from_file('../output/trades/rsi/2023/allow_overlap-default.xlsx', 'bear')
test_result_bull = OptionsMultiTradesResult.load_from_file('../output/trades/rsi/2023/allow_overlap-default.xlsx', 'bull')

In [202]:
overbought_stocks = pd.read_csv('../output/trades/rsi/2022-h1/rsi-crossed-above.csv')
overbought_stocks[[DerivativesBaseColumns.Identifier, DerivativesBaseColumns.Date]].head(1)

,Identifier,Date
0,PIDILITIND,2022-01-21


In [23]:
symbol = 'HINDZINC'
signal_date = '2016-02-01'

In [220]:
disp_cols = [DerivativesBaseColumns.Identifier, DerivativesBaseColumns.Date, 
             DerivativesBaseColumns.StrikePrice, DerivativesBaseColumns.ExpiryDate, DerivativesBaseColumns.OpenInterest,
             DerivativesCalculatedColumns.LotSize,
             DerivativesCalculatedColumns.CloseAmount, DerivativesCalculatedColumns.LowAmount, DerivativesCalculatedColumns.HighAmount]
def show_available_entry_options(symbol, option_type, signal_date, data, skip_price_check = False, skip_expiry_check = False, skip_oi_check = False):
    date_range = [pd.to_datetime(signal_date), pd.to_datetime(signal_date) + datetime.timedelta(days=7)]
    return data[
        (data[DerivativesBaseColumns.Identifier] == symbol)
        &
        (data[DerivativesBaseColumns.OptionType] == option_type)
        &
        (data[DerivativesBaseColumns.Date] == date_range[0])
        &
        (
            (data[DerivativesCalculatedColumns.CloseAmount].between(8000, 12000))
            |
            (skip_price_check == True)
        ) 
        &
        (
            (pd.to_datetime(data[DerivativesBaseColumns.ExpiryDate]) >= date_range[1])
            |
            (skip_expiry_check == True)
        )
        &
        (
            (data[DerivativesBaseColumns.OpenInterest] >= (data[DerivativesCalculatedColumns.LotSize] * 50))
            |
            (skip_oi_check == True)
        )
            
    ].sort_values([DerivativesCalculatedColumns.CloseAmount])[disp_cols]

def show_available_exit_options(symbol, option_type, strike, expiry, signal_date, data, holding_days = 7):
    date_range = [pd.to_datetime(signal_date), pd.to_datetime(signal_date) + datetime.timedelta(days=holding_days)]
    return data[
        (data[DerivativesBaseColumns.Identifier] == symbol)
        &
        (data[DerivativesBaseColumns.OptionType] == option_type)
        &
        (data[DerivativesBaseColumns.Date].between(date_range[0], date_range[1]))
        &
        (data[DerivativesBaseColumns.StrikePrice] == strike)
        &
        (data[DerivativesBaseColumns.ExpiryDate] == expiry)
    ].sort_values([DerivativesBaseColumns.Date])[disp_cols]

In [227]:
overbought_stocks.head(15)[['Identifier', 'Date', 'Close']]

,Identifier,Date,Close
0,PIDILITIND,2022-01-21,2700.60
1,TRENT,2022-01-21,1181.85
2,AUBANK,2022-01-21,1256.70
3,INDHOTEL,2022-01-21,206.35
4,HAL,2022-01-21,1418.15
5,HEROMOTOCO,2022-01-25,2782.05
6,CHOLAFIN,2022-01-27,654.90
7,BANDHANBNK,2022-01-28,316.95
8,NTPC,2022-01-31,142.05
9,BANKBARODA,2022-01-31,107.55


In [247]:
test_data = data_2022_h1
symbols = overbought_stocks[DerivativesBaseColumns.Identifier].unique()
test_data = test_data[test_data[DerivativesBaseColumns.Identifier].isin(symbols)]

In [249]:
take_index = 7
symbol = overbought_stocks[DerivativesBaseColumns.Identifier].values[take_index]
signal_date = overbought_stocks[DerivativesBaseColumns.Date].values[take_index]

show_available_entry_options(symbol, 'PE', signal_date, test_data,
    skip_price_check = False,
    skip_oi_check = False,
    skip_expiry_check = False
  ).sort_values(DerivativesBaseColumns.OpenInterest, ascending=False)

,Identifier,Date,StrkPric,ExpiryDate,OpenInterest,LotSize,CloseAmount,LowAmount,HighAmount
1295085,BANDHANBNK,2022-01-28,280.0,24-Feb-2022,408600,2500.0,8875.0,7375.0,9625.0
1295086,BANDHANBNK,2022-01-28,285.0,24-Feb-2022,334800,2500.0,10500.0,8875.0,11500.0


###### Entry

In [230]:
show_available_exit_options('BANDHANBNK', 'PE', 280.0, '24-Feb-2022', '2022-01-28', data_2022_h1)

,Identifier,Date,StrkPric,ExpiryDate,OpenInterest,LotSize,CloseAmount,LowAmount,HighAmount
1295085,BANDHANBNK,2022-01-28,280.0,24-Feb-2022,408600,2500.0,8875.0,7375.0,9625.0
1364936,BANDHANBNK,2022-01-31,280.0,24-Feb-2022,412200,2500.0,7625.0,6875.0,8875.0
1434847,BANDHANBNK,2022-02-01,280.0,24-Feb-2022,392400,2500.0,5625.0,5125.0,7750.0
1504836,BANDHANBNK,2022-02-02,280.0,24-Feb-2022,381600,2500.0,4750.0,4375.0,6125.0
1575095,BANDHANBNK,2022-02-03,280.0,24-Feb-2022,378000,2500.0,5125.0,4000.0,5125.0
1645066,BANDHANBNK,2022-02-04,280.0,24-Feb-2022,345600,2500.0,4750.0,4500.0,5625.0


In [231]:
buy = bear_trade_builder.get_entry_option(data_2022_h1, symbol, signal_date, 0)
sell = bear_trade_builder.get_exit_option_for_buy_option(data_2022_h1, buy, 0)

In [233]:
sell

,Identifier,StrkPric,ExpiryDate,OptionType,Date,Credit,Quantity,Type
1,BANDHANBNK,290.0,24-Feb-2022,PE,2022-02-02,6750.0,-2500.0,Sell


###### Exit

In [ ]:
data[
        (data[DerivativesBaseColumns.Identifier] == 'OBEROIRLTY')
        & (data[DerivativesBaseColumns.ExpiryDate] == '25-May-2023')
        & (data[DerivativesBaseColumns.StrikePrice] == 960)
        & (data[DerivativesBaseColumns.OptionType] == 'PE')
        & (data[DerivativesBaseColumns.Date].between(pd.to_datetime('2023-05-16') + datetime.timedelta(days=1), '2023-05-23'))
        & (data[DerivativesBaseColumns.OpenInterest] >= (data[DerivativesCalculatedColumns.LotSize] * 25))
    ][[DerivativesBaseColumns.Date, DerivativesBaseColumns.OpenInterest, DerivativesCalculatedColumns.HighAmount, DerivativesCalculatedColumns.LotSize]]

,Date,OpenInterest,HighAmount,LotSize
4335957,2023-05-17,32200.0,36575.0,700.0
4379353,2023-05-18,25900.0,50400.0,700.0
4414443,2023-05-19,22400.0,44660.0,700.0
4498644,2023-05-22,22400.0,29085.0,700.0
4532135,2023-05-23,22400.0,26285.0,700.0
